# Работа с базами данных (2)

__Автор задач: Блохин Н.В. (NVBlokhin@fa.ru)__

Материалы:
* Макрушин С.В. Лекция "Работа с базами данных"
* https://sqliteonline.com/
* https://docs.python.org/3/library/sqlite3.html
* https://www.sqlitetutorial.net/sqlite-index/
* https://docs.python.org/3/library/sqlite3.html#sqlite3.IntegrityError
* https://www.sqlitetutorial.net/sqlite-alter-table/
* https://www.sqlitetutorial.net/sqlite-create-view/
* https://habr.com/ru/post/664000/
* https://learnsql.com/blog/what-is-common-table-expression/


## Задачи для совместного разбора

In [1]:
import pandas as pd
import sqlite3

# данные
students = pd.DataFrame(
    [
        ("Сотников Евгений Янович", 1),
        ("Степанова Виктория Константиновна", 1),
        ("Горелова Вероника Яновна", 2),
        ("Гришин Иван Романович", 3),
    ],
    columns=["name", "group_id"],
)
groups = list(zip([1, 2, 3], ["ПМ20-1", "ПМ20-2", "ПМ20-3"]))

con = sqlite3.connect("demo.sqlite")
con.execute("PRAGMA foreign_keys = 1")
cur = con.cursor()

# создаем таблицы
sql = """
DROP TABLE IF EXISTS StudentGroup;
DROP TABLE IF EXISTS Student;
CREATE TABLE StudentGroup (
    id int PRIMARY KEY,
    name varchar
);

CREATE TABLE Student(
    name VARCHAR PRIMARY KEY,
    group_id INT,
    FOREIGN KEY (group_id) REFERENCES StudentGroup(id)
);
"""
cur.executescript(sql)
con.commit()

# добавляем записи
sql = """
INSERT INTO StudentGroup(id, name) VALUES (?, ?)
"""
cur.executemany(sql, groups)
con.commit()

students.to_sql("Student", con, if_exists="append", index=False)

4

1\. Добавить столбец Age со значением по умолчанию. Добавить запись к таблицу

In [2]:
sql = '''
ALTER TABLE Student
    ADD COLUMN Age INT DEFAULT 18
'''
cur.execute(sql)
con.commit()

pd.read_sql_query("SELECT * FROM Student", con)

sql = '''
INSERT INTO Student(name, group_id, Age)
    VALUES ("Астахов Владислав Игоревич", 3, 20)
'''
try:
    cur.execute(sql)
except sqlite3.OperationalError as e:
    print("Operational error: ", e)
else:
    con.commit()

pd.read_sql_query("SELECT * FROM Student", con)

,name,group_id,Age
0,Сотников Евгений Янович,1,18
1,Степанова Виктория Константиновна,1,18
2,Горелова Вероника Яновна,2,18
3,Гришин Иван Романович,3,18
4,Астахов Владислав Игоревич,3,20


2\. Занумеруйте студентов в рамках каждой группы.

In [3]:
pd.read_sql_query("""
    SELECT name,
           group_id,
           ROW_NUMBER() OVER(PARTITION BY group_id ORDER BY name) as rid
    FROM Student
""", con)

,name,group_id,rid
0,Сотников Евгений Янович,1,1
1,Степанова Виктория Константиновна,1,2
2,Горелова Вероника Яновна,2,1
3,Астахов Владислав Игоревич,3,1
4,Гришин Иван Романович,3,2


3\. Выведите уникальные номера студентов

In [4]:
pd.read_sql_query("""
    SELECT DISTINCT rid
    FROM (
        SELECT name,
               group_id,
               ROW_NUMBER() OVER(PARTITION BY group_id ORDER BY name) as rid
        FROM Student
    )
""", con)

,rid
0,1
1,2


In [5]:
sql = """
CREATE VIEW StudentView AS 
    SELECT name,
           group_id,
           ROW_NUMBER() OVER(PARTITION BY group_id ORDER BY name) as rid
    FROM Student
"""
cur.execute(sql)
con.commit()

In [6]:
pd.read_sql_query("""
    SELECT DISTINCT rid
    FROM StudentView
""", con)

,rid
0,1
1,2


In [7]:
pd.read_sql_query("""
    WITH StudentCTE AS (
        SELECT name,
               group_id,
               ROW_NUMBER() OVER(PARTITION BY group_id ORDER BY name) as rid
        FROM Student
    )
    
    SELECT DISTINCT rid
    FROM StudentCTE
""", con)

,rid
0,1
1,2


In [9]:
pd.read_sql_query("""
    SELECT DISTINCT rid
    FROM StudentCTE
""", con)

DatabaseError: Execution failed on sql '
    SELECT DISTINCT rid
    FROM StudentCTE
': no such table: StudentCTE

## Лабораторная работа 4

__При решении данных задач не подразумевается использования циклов или генераторов Python в ходе работы с пакетами `numpy` и `pandas`, если в задании не сказано обратного. Решения задач, в которых для обработки массивов `numpy` или структур `pandas` используются явные циклы (без согласования с преподавателем), могут быть признаны некорректными и не засчитаны.__

__Для начала работы подключитесь к БД `recipes.db` и создайте объект-курсор.__

In [1]:
import pandas as pd
import sqlite3

In [2]:
con = sqlite3.connect("recipes.db")
cur = con.cursor()

<p class="task" id="1"></p>

1\. Создайте уникальный индекс для таблицы `Review` для обеспечения уникальности сочетания значений в полях `user_id` и `recipe_id`. 

In [3]:
df = pd.read_sql_query('''
SELECT * FROM Review
''', con)
df

,id,user_id,recipe_id,date,rating,review
0,370476,21752,57993,2003-05-01,5,Last week whole sides of frozen salmon fillet ...
1,624300,431813,142201,2007-09-16,5,So simple and so tasty! I used a yellow capsi...
2,187037,400708,252013,2008-01-10,4,"Very nice breakfast HH, easy to make and yummy..."
3,706134,2001852463,404716,2017-12-11,5,These are a favorite for the holidays and so e...
4,312179,95810,129396,2008-03-14,5,Excellent soup! The tomato flavor is just gre...
...,...,...,...,...,...,...
126691,1013457,1270706,335534,2009-05-17,4,This recipe was great! I made it last night. I...
126692,158736,2282344,8701,2012-06-03,0,This recipe is outstanding. I followed the rec...
126693,1059834,689540,222001,2008-04-08,5,"Well, we were not a crowd but it was a fabulou..."
126694,453285,2000242659,354979,2015-06-02,5,I have been a steak eater and dedicated BBQ gr...


In [4]:
pd.read_sql_query("""
    SELECT id, user_id, recipe_id, date, rating, review,
           ROW_NUMBER() OVER(PARTITION BY recipe_id ORDER BY user_id) as rid
    FROM Review
""", con)

,id,user_id,recipe_id,date,rating,review,rid
0,532498,32421,48,2002-03-15,0,The flavor was great. But I think there was no...,1
1,532499,68674,48,2004-05-03,2,I picked this recipe over the other BCP recipe...,2
2,462144,53959,55,2006-01-12,4,I liked it. I was surprised since it didn't ha...,1
3,462145,165567,55,2006-03-31,5,I LOVED this recipe! I was looking for a guaca...,2
4,462147,851190,55,2010-05-23,5,I used Italian plum tomatoes for this as they ...,3
...,...,...,...,...,...,...,...
126691,744576,1925885,536729,2018-08-16,5,"Loved this simple salad! DH saw it and said, &...",4
126692,631003,1460111,536747,2018-09-26,0,Very cute cake! Love the marshmallow pom poms....,1
126693,631001,2001285346,536747,2018-09-06,0,Such an art Beautiful,2
126694,631000,2001996022,536747,2018-09-06,0,THANK YOUUUU!!! I'm dying waiting for this tut...,3


<p class="task" id="2"></p>

2\. Напишите функцию `add_review(review_id, user_id, recipe_id, date, rating, review)`, которая добавляет запись в таблицу `Review`. В случае успешного добавления функция должна вернуть значение 0. В случае нарушения ограничения целостности функция должна вернуть значение 1. В случае любых других ошибок функция должна вернуть значение 2. Продемонстрируйте работу функции, попытавшись добавить одну и ту же запись дважды в двух ячейках подряд.

Для решения задачи воспользуйтесь механизмом try - except и обработайте соответствующее исключение.

In [5]:
def add_review(review_id, user_id, recipe_id, date, rating, review):
    sql = '''
    INSERT INTO Review (id, user_id, recipe_id, date, rating, review)  
        VALUES (?, ?, ?, ?, ?, ?);
    '''
    
    data_tuple = (review_id, user_id, recipe_id, date, rating, review)
    
    try:
        cur.execute(sql, data_tuple)
        print(0)
    except sqlite3.IntegrityError:
        print(1)
    except:
        print(2)
    else: 
        con.commit()

In [6]:
review_id = int(input('Введите id отзыва: '))
user_id = int(input('Введите id пользователя: '))
recipe_id = int(input('Введите id рецепта: '))
date = input('Введите дату отзыва в формате год-месяц-число: ')
rating = int(input('Введите оценку рецепта: '))
review = input('Введите свой отзыв: ')

add_review(review_id, user_id, recipe_id, date, rating, review)

Введите id отзыва: 111
Введите id пользователя: 222
Введите id рецепта: 333
Введите дату отзыва в формате год-месяц-число: 2023-03-16
Введите оценку рецепта: 5
Введите свой отзыв: Вкусно
0


In [7]:
review_id = int(input('Введите id отзыва: '))
user_id = int(input('Введите id пользователя: '))
recipe_id = int(input('Введите id рецепта: '))
date = input('Введите дату отзыва в формате год-месяц-число: ')
rating = int(input('Введите оценку рецепта: '))
review = input('Введите свой отзыв: ')

add_review(review_id, user_id, recipe_id, date, rating, review)

Введите id отзыва: 111
Введите id пользователя: 222
Введите id рецепта: 333
Введите дату отзыва в формате год-месяц-число: 2023-03-16
Введите оценку рецепта: 5
Введите свой отзыв: Вкусно
1


In [8]:
df = pd.read_sql_query('''
SELECT * FROM Review
''', con)
df

,id,user_id,recipe_id,date,rating,review
0,370476,21752,57993,2003-05-01,5,Last week whole sides of frozen salmon fillet ...
1,624300,431813,142201,2007-09-16,5,So simple and so tasty! I used a yellow capsi...
2,187037,400708,252013,2008-01-10,4,"Very nice breakfast HH, easy to make and yummy..."
3,706134,2001852463,404716,2017-12-11,5,These are a favorite for the holidays and so e...
4,312179,95810,129396,2008-03-14,5,Excellent soup! The tomato flavor is just gre...
...,...,...,...,...,...,...
126692,158736,2282344,8701,2012-06-03,0,This recipe is outstanding. I followed the rec...
126693,1059834,689540,222001,2008-04-08,5,"Well, we were not a crowd but it was a fabulou..."
126694,453285,2000242659,354979,2015-06-02,5,I have been a steak eater and dedicated BBQ gr...
126695,691207,463435,415599,2010-09-30,5,Wonderful and simple to prepare seasoning blen...


<p class="task" id="3"></p>

3\. _Измените_ таблицу Review, добавив в нее поле `toxic` булева типа. 

In [9]:
sql = '''
ALTER TABLE Review ADD COLUMN toxic BOOL
'''
cur.execute(sql)

In [10]:
df = pd.read_sql_query('''
SELECT * FROM Review
''', con)
df

,id,user_id,recipe_id,date,rating,review,toxic
0,370476,21752,57993,2003-05-01,5,Last week whole sides of frozen salmon fillet ...,None
1,624300,431813,142201,2007-09-16,5,So simple and so tasty! I used a yellow capsi...,None
2,187037,400708,252013,2008-01-10,4,"Very nice breakfast HH, easy to make and yummy...",None
3,706134,2001852463,404716,2017-12-11,5,These are a favorite for the holidays and so e...,None
4,312179,95810,129396,2008-03-14,5,Excellent soup! The tomato flavor is just gre...,None
...,...,...,...,...,...,...,...
126692,158736,2282344,8701,2012-06-03,0,This recipe is outstanding. I followed the rec...,None
126693,1059834,689540,222001,2008-04-08,5,"Well, we were not a crowd but it was a fabulou...",None
126694,453285,2000242659,354979,2015-06-02,5,I have been a steak eater and dedicated BBQ gr...,None
126695,691207,463435,415599,2010-09-30,5,Wonderful and simple to prepare seasoning blen...,None


<p class="task" id="4"></p>

4\. Вам дан классификатор `clf`, который классифицирует тексты отзывов как токсичные (`True`) и не токсичные (`False`).
Напишите функцию `classify_reviews`, которая итеративно получает пакет (батч) `batch_size` строк из таблицы Reviews, у которых не проставлено значение в столбце `toxic`, делает для них прогноз при помощи модели `clf` и обновляет соответствующие строки в БД. Данная процедура выполняется до тех пор, пока в БД есть строки, для которых требуется получить прогноз.

Продемонстрируйте результат, выведя на экран количество токсичных и не токсичных отзывов в таблице.

In [11]:
from sklearn.dummy import DummyClassifier

clf = DummyClassifier(strategy="uniform").fit(None, [True, False])

In [13]:
def classify_reviews():

IndentationError: expected an indented block (968733836.py, line 1)

<p class="task" id="5"></p>

5\. Создайте представление `RecipeWithYear`, в котором добавлен дополнительный столбец `year`, содержащий год даты из столбца `submitted`. Сделайте выборку из этого представления и выведите на экран количество рецептов с разбивкой по годам.

In [7]:
sql = '''
CREATE VIEW RecipeWithYear AS
SELECT *, strftime('%Y', date) as year FROM Review
'''
cur.executescript(sql)

In [8]:
df1 = pd.read_sql_query('''
SELECT * FROM RecipeWithYear
''', con)
df1

,id,user_id,recipe_id,date,rating,review,year
0,370476,21752,57993,2003-05-01,5,Last week whole sides of frozen salmon fillet ...,2003
1,624300,431813,142201,2007-09-16,5,So simple and so tasty! I used a yellow capsi...,2007
2,187037,400708,252013,2008-01-10,4,"Very nice breakfast HH, easy to make and yummy...",2008
3,706134,2001852463,404716,2017-12-11,5,These are a favorite for the holidays and so e...,2017
4,312179,95810,129396,2008-03-14,5,Excellent soup! The tomato flavor is just gre...,2008
...,...,...,...,...,...,...,...
126691,1013457,1270706,335534,2009-05-17,4,This recipe was great! I made it last night. I...,2009
126692,158736,2282344,8701,2012-06-03,0,This recipe is outstanding. I followed the rec...,2012
126693,1059834,689540,222001,2008-04-08,5,"Well, we were not a crowd but it was a fabulou...",2008
126694,453285,2000242659,354979,2015-06-02,5,I have been a steak eater and dedicated BBQ gr...,2015


In [9]:
sql = '''
SELECT year, COUNT(year) FROM RecipeWithYear
GROUP BY year
'''

df2 = pd.read_sql_query(sql, con)
df2

,year,COUNT(year)
0,2000,13
1,2001,305
2,2002,2494
3,2003,3879
4,2004,5070
5,2005,7577
6,2006,9964
7,2007,15634
8,2008,18614
9,2009,17672


<p class="task" id="6"></p>

6\. Напишите запрос на языке SQL, который возвращает все строки из таблицы `Recipe` с дополнительным столбцом, содержащем номер рецепта. Рецепты нумеруются целыми числами, начиная с 1, в __рамках каждого года__ в порядке их добавления в БД (столбец `submitted`). Получите результат в виде `pd.DataFrame`. Посчитайте и выведите на экран количество строк полученного `pd.DataFrame`, для которых сгенерированный номер кратен 50.

In [11]:
df3 = pd.read_sql_query("""
    SELECT *, ROW_NUMBER() OVER(PARTITION BY Year ORDER BY date) as rid
    FROM RecipeWithYear
""", con)
df3

,id,user_id,recipe_id,date,rating,review,year,rid
0,901198,2156,148,2000-06-02,0,Would someone please check the Nutrition Facts...,2000,1
1,645250,3114,153,2000-10-30,4,Took about 70 minutes to bake.,2000,2
2,640206,3228,288,2000-11-08,3,tasty. you can substitute granulated sugar for...,2000,3
3,375482,3556,19270,2000-11-22,4,I first got this recipe about 25 years ago fro...,2000,4
4,385704,3504,4048,2000-11-23,0,makes 2- 9inch pies or 1 deep dish pie,2000,5
...,...,...,...,...,...,...,...,...
126691,691137,2001563476,431399,2018-12-18,5,I saw a few so so reviews but needed a small b...,2018,2327
126692,1076676,2950250,329804,2018-12-18,5,This is the BEST chocolate syrup ever!!. It re...,2018,2328
126693,630476,2002370666,13598,2018-12-18,0,i love this recipes make it easy for everyone ...,2018,2329
126694,691138,2002371792,431399,2018-12-19,4,"These cookies taste wonderful, but 14 minutes ...",2018,2330


In [12]:
len(df3[df3['rid']%50 == 0])

2524

<p class="task" id="7"></p>

7\. Используя обобщенное табличное выражение и решение задачи 6, напишите запрос на языке SQL, который вернет количество строк, для которых сгенерированный номер кратен 50. Выполните запрос и выведите количество таких строк на экран.

In [14]:
sql = '''
WITH Zapr AS
    (SELECT *, ROW_NUMBER() OVER(PARTITION BY Year) AS rid FROM RecipeWithYear)
    
SELECT COUNT(*)
FROM Zapr
WHERE rid%50=0
'''
pd.read_sql_query(sql, con)

,COUNT(*)
0,2524
